# Custom Layers

This exercise is adapted from [this repository](https://github.com/david-drew/OpenVINO-Custom-Layers).

Note that the classroom workspace is running OpenVINO 2019.r3, while this exercise was originally created for 2019.r2. This exercise will work appropriately in the workspace, but there may be some other differences you need to account for if you use a custom layer yourself.

The below steps will walk you through the full walkthrough of creating a custom layer; as such, there is not a related solution video. Note that custom layers is an advanced topic, and one that is not expected to be used often (if at all) in most use cases of the OpenVINO toolkit. This exercise is meant to introduce you to the concept, but you won't need to use it again in the rest of this course.

<hr>

# Example Custom Layer: The Hyperbolic Cosine (cosh) Function
We will follow the steps involved for implementing a custom layer using the simple hyperbolic cosine (cosh) function. The cosh function is mathematically calculated as:
> `cosh(x) = (e^x + e^-x) / 2`

### Build the Model
First, export the below paths to shorten some of what you need to enter later:

```sh
export CLWS=/home/workspace/cl_tutorial
export CLT=$CLWS/OpenVINO-Custom-Layers
```

Then run the following to create the TensorFlow model including the cosh layer.

```sh
mkdir $CLWS/tf_model
python $CLT/create_tf_model/build_cosh_model.py $CLWS/tf_model
```
You should receive a message similar to:

```sh
Model saved in path: /tf_model/model.ckpt
```

<hr>

# Creating the <i>cosh</i> Custom Layer
### Generate the Extension Template Files Using the Model Extension Generator

We will use the Model Extension Generator tool to automatically create templates for all the extensions needed by the Model Optimizer to convert and the Inference Engine to execute the custom layer. The extension template files will be partially replaced by Python and C++ code to implement the functionality of `cosh` as needed by the different tools. To create the four extensions for the `cosh` custom layer, we run the Model Extension Generator with the following options:

- `--mo-tf-ext` = Generate a template for a Model Optimizer TensorFlow extractor
- `--mo-op` = Generate a template for a Model Optimizer custom layer operation
- `--ie-cpu-ext` = Generate a template for an Inference Engine CPU extension
- `--ie-gpu-ext` = Generate a template for an Inference Engine GPU extension
- `--output_dir` = set the output directory. Here we are using `$CLWS/cl_cosh` as the target directory to store the output from the Model Extension Generator.

To create the four extension templates for the `cosh` custom layer, given we are in the `$CLWS` directory, we run the command:
```sh
mkdir cl_cosh
```
<hr>

```sh
python /opt/intel/openvino/deployment_tools/tools/extension_generator/extgen.py new --mo-tf-ext --mo-op --ie-cpu-ext --ie-gpu-ext --output_dir=$CLWS/cl_cosh
```

The Model Extension Generator will start in interactive mode and prompt us with questions about the custom layer to be generated. Use the text between the `[]`'s to answer each of the Model Extension Generator questions as follows:

```
Enter layer name: 
[cosh]

Do you want to automatically parse all parameters from the model file? (y/n)
...
[n]

Enter all parameters in the following format:
...
Enter 'q' when finished:
[q]

Do you want to change any answer (y/n) ? Default 'no'
[n]

Do you want to use the layer name as the operation name? (y/n)
[y]

Does your operation change shape? (y/n)  
[n]

Do you want to change any answer (y/n) ? Default 'no'
[n]
```
<hr>

When complete, the output text will appear similar to:

```
Stub file for TensorFlow Model Optimizer extractor is in /home/<user>/cl_tutorial/cl_cosh/user_mo_extensions/front/tf folder
Stub file for the Model Optimizer operation is in /home/<user>/cl_tutorial/cl_cosh/user_mo_extensions/ops folder
Stub files for the Inference Engine CPU extension are in /home/<user>/cl_tutorial/cl_cosh/user_ie_extensions/cpu folder
Stub files for the Inference Engine GPU extension are in /home/<user>/cl_tutorial/cl_cosh/user_ie_extensions/gpu folder
```
<hr>
Template files (containing source code stubs) that may need to be edited have just been created in the following locations:


<b>TensorFlow Model Optimizer extractor extension:</b>
    - $CLWS/cl_cosh/user_mo_extensions/front/tf/
    - cosh_ext.py
    
<b>Model Optimizer operation extension:</b>
     - $CLWS/cl_cosh/user_mo_extensions/ops
     - cosh.py
     
<b>Inference Engine CPU extension:</b>
     - $CLWS/cl_cosh/user_ie_extensions/cpu
     - ext_cosh.cpp
     - CMakeLists.txt
     
<b>Inference Engine GPU extension:</b>
     - $CLWS/cl_cosh/user_ie_extensions/gpu
     - cosh_kernel.cl
     - cosh_kernel.xml
     
<hr>

Instructions on editing the template files are provided in later parts of this tutorial.
For reference, or to copy to make the changes quicker, pre-edited template files are provided by the tutorial in the `$CLT` directory.

# Using Model Optimizer to Generate IR Files Containing the Custom Layer
We will now use the generated extractor and operation extensions with the Model Optimizer to generate the model IR files needed by the Inference Engine. The steps covered are:

    1. Edit the extractor extension template file (already done - we will review it here)
    2. Edit the operation extension template file (already done - we will review it here)
    3. Generate the Model IR Files